# 行銷活動效益知多少?顧客回購了嗎?－建立會員消費紀錄資料庫(附Python程式碼)-系列2

### 作者：張友志(臺灣行銷研究特邀作者)

[系列文章鏈結](https://reurl.cc/k0aQyK/ "Title")

In [7]:
# 引入套件
import pandas as pd
import numpy as np
from datetime import timedelta, datetime, tzinfo, timezone

## 一、取出分析資料（系列 1－2018年度）

In [8]:
# 將2018年原始資料按照1~12月分類
# 先設立一個可以放2018年每個月資料的list
original_2018 = []
for i in range(1,13):
    # 當月份等於12月的時候，執行下方程式
    if i == 12 :
        # 設立選取資料起始時間 (2018年 12月 1日)
        begin = datetime(2018, i, 1)
        # 設立選取資料結束時間 (2019年 1月 1日)
        end = datetime(2019, 1, 1)
        # 選取資料 ( 結束時間之前 )
        pd1_2018 = product_1[product_1['訂單時間'] <= end]
        # 選取資料 ( 開始時間之後 )
        pd1_2018 = pd1_2018[pd1_2018['訂單時間'] >= begin]
        # 將12月的資料加到存放每個月資料的list中
        original_2018.append(pd1_2018)
    # 當月份不等於12月的時候，執行下方程式
    else :
        # 設立選取資料起始時間 (2018年 某月 1日)
        begin = datetime(2018, i, 1)
        # 設立選取資料結束時間 (2018年 某月+1個月 1日)
        end = datetime(2018, i+1, 1)
        # 選取資料 ( 結束時間之前 )
        pd1_2018 = product_1[product_1['訂單時間'] <= end]
        # 選取資料 ( 開始時間之後 )
        pd1_2018 = pd1_2018[pd1_2018['訂單時間'] >= begin]
        # 將該月份的資料加到存放每個月資料的list中
        original_2018.append(pd1_2018)

## 二、建立會員消費紀錄資料庫

In [9]:
# 建立每個會員的消費紀錄資料庫
customer_record = {}
for mon in range(len(original_2018)):
    # 將每個訂單的利潤計算出來存成list
    profit_gp = original_2018[mon].groupby('訂單時間')["利潤"].sum().tolist()
    # 把訂單時間重複的drop掉
    original_2018[mon] =  original_2018[mon].drop_duplicates(subset=['訂單時間'])
    # 把利潤補回原本df
    original_2018[mon]['利潤'] = profit_gp
    # 將index 重新排序
    original_2018[mon] = original_2018[mon].reset_index(drop = True)
    a = original_2018[mon]
    for i in range(len(a['會員'])):
        # 如果會員「不在」字典的keys中，執行下方程式
        if str(a['會員'][i]) not in customer_record.keys():
            # 新增會員到keys中，並且放上對應的消費紀錄
            customer_record[ str(a['會員'][i]) ] = {'1':str(mon+1)+'月_'+a['廣告代號'][i]} 
        # 如果會員已經在字典的keys中，執行下方程式
        else:
            # 直接在原有的會員keys下，放上對應的消費紀錄
            customer_record[ str(a['會員'][i]) ][str(len(customer_record[ str(a['會員'][i]) ])+1)] = str(mon+1)+'月_'+a['廣告代號'][i]